<a href="https://colab.research.google.com/github/rikkudo/rikkudo/blob/main/acfm_ho3d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment Setting**

In [1]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/jaimergp/miniforge/releases/latest/download/Mambaforge-colab-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:48
🔁 Restarting kernel...


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

print(torch.__version__)
print(torch.version.cuda)

1.11.0+cu113
11.3


In [2]:
!gdown "12hDV1nyrC5M56d7jlLVVq4rlVgDWMixZ"
!conda env update --file environment_modified.yml

Downloading...
From: https://drive.google.com/uc?id=12hDV1nyrC5M56d7jlLVVq4rlVgDWMixZ
To: /content/environment_modified.yml
100% 6.03k/6.03k [00:00<00:00, 11.4MB/s]
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / 

**Get ACFM Code**

In [3]:
!git clone https://github.com/fkokkinos/acfm_video_3d_reconstruction acfm

Cloning into 'acfm'...
remote: Enumerating objects: 201, done.
remote: Total 201 (delta 0), reused 0 (delta 0), pack-reused 201
Receiving objects: 100% (201/201), 186.67 MiB | 14.21 MiB/s, done.
Resolving deltas: 100% (36/36), done.
Checking out files: 100% (173/173), done.


In [4]:
%cd "/content/acfm/multiframe/data/optical_flow/model/correlation_package"
!python -W ignore setup.py install

/content/acfm/multiframe/data/optical_flow/model/correlation_package
running install
running bdist_egg
running egg_info
creating correlation_cuda.egg-info
writing correlation_cuda.egg-info/PKG-INFO
writing dependency_links to correlation_cuda.egg-info/dependency_links.txt
writing top-level names to correlation_cuda.egg-info/top_level.txt
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
reading manifest file 'correlation_cuda.egg-info/SOURCES.txt'
writing manifest file 'correlation_cuda.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'correlation_cuda' extension
creating /content/acfm/multiframe/data/optical_flow/model/correlation_package/build
creating /content/acfm/multiframe/data/optical_flow/model/correlation_package/build/temp.linux-x86_64-3.8
Emitting ninja build file /content/acfm/multiframe/data/optical_flow/model/correlation_package/build/temp.linux-x86_64-3.8/build.ninja...
Compiling obje

In [5]:
!pip install "git+https://github.com/facebookresearch/pytorch3d.git"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/pytorch3d.git to /tmp/pip-req-build-jc797bqt
  Created wheel for pytorch3d: filename=pytorch3d-0.6.2-cp38-cp38-linux_x86_64.whl size=25115111 sha256=5ae3592dce786bb3d365438f5d856b77121d16736a377f5be0c366d91fd951c8
  Stored in directory: /tmp/pip-ephem-wheel-cache-yz3rxnjw/wheels/5e/a7/7c/814c3888eaaecaaf0f533faeb3c6812d82274484be51f7441c
Successfully built pytorch3d


**Train Dataset**

In [6]:
%cd "/content/acfm/multiframe/misc"
!gdown 1rsoy4SIgFCP_vy0k-1GJzI7foFk3RnsH
!7z t cachedir.7z

/content/acfm/multiframe/misc
Downloading...
From: https://drive.google.com/uc?id=1rsoy4SIgFCP_vy0k-1GJzI7foFk3RnsH
To: /content/acfm/multiframe/misc/cachedir.7z
100% 856M/856M [00:17<00:00, 49.6MB/s]

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 856491929 bytes (817 MiB)

Testing archive: cachedir.7z
--
Path = cachedir.7z
Type = 7z
Physical Size = 856491929
Headers Size = 1341
Method = LZMA2:24
Solid = +
Blocks = 1

  0%      0% 25 T cachedir/cub/data/test_cub_cleaned.mat                                                  1% 26 T cachedir/cub/data/testval_cub_cleaned.mat                                        

In [18]:
%cd "/content/acfm/multiframe/data"

/content/acfm/multiframe/data


In [20]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 128 kB 4.0 MB/s 
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-any.whl size=14759 sha256=c81e47621016b7f98af96340652f1823ecf6bae4ce5f0637dd5353345540146a
  Stored in directory: /tmp/pip-ephem-wheel-cache-mr89mf5g/wheels/7b/7b/5d/656f46cd6889e4c93977be9586901d0adc1271b2d876c84c96
Successfully built gdown


In [23]:
!gdown '1bWx893_bBeH3ar1T3bwIQuX-Kqj2JSFV'

Downloading...
From: https://drive.google.com/uc?id=1bWx893_bBeH3ar1T3bwIQuX-Kqj2JSFV
To: /content/acfm/multiframe/data/h2o3d_v1.zip
100% 15.8G/15.8G [04:35<00:00, 57.5MB/s]


In [24]:
!unzip h2o3d_v1.zip

Streaming output truncated to the last 5000 lines.
  inflating: train/SSBC1/meta/0132.pkl  
  inflating: train/SSBC1/meta/0654.pkl  
  inflating: train/SSBC1/meta/0867.pkl  
  inflating: train/SSBC1/meta/0873.pkl  
  inflating: train/SSBC1/meta/0683.pkl  
  inflating: train/SSBC1/meta/0697.pkl  
  inflating: train/SSBC1/meta/0696.pkl  
  inflating: train/SSBC1/meta/0682.pkl  
  inflating: train/SSBC1/meta/0872.pkl  
  inflating: train/SSBC1/meta/0866.pkl  
  inflating: train/SSBC1/meta/0133.pkl  
  inflating: train/SSBC1/meta/0655.pkl  
  inflating: train/SSBC1/meta/0641.pkl  
  inflating: train/SSBC1/meta/0899.pkl  
  inflating: train/SSBC1/meta/0127.pkl  
  inflating: train/SSBC1/meta/0669.pkl  
  inflating: train/SSBC1/meta/0494.pkl  
  inflating: train/SSBC1/meta/0480.pkl  
  inflating: train/SSBC1/meta/0457.pkl  
  inflating: train/SSBC1/meta/0331.pkl  
  inflating: train/SSBC1/meta/0325.pkl  
  inflating: train/SSBC1/meta/0443.pkl  
  inflating: train/SSBC1/meta/0319.pkl  
  infl

In [26]:
%cd "/content/acfm/multiframe"

!CUDA_VISIBLE_DEVICES=0 python -B main.py --name=horse_net_kp --category horse --display_port 8097 --batch_size=12 --learning_rate 1e-4 --num_lbs 16 --nz_feat 256 --symmetric_texture=False --symmetric=False --mesh_dir meshes/horse_aligned.obj --rigid_wt 10. --root_dir /content/acfm/multiframe/data/train --tmp_dir tmp_horse_sfm/ --of_loss_wt 0.1 --mask_loss_wt 2. --boundaries_reg_wt 1. --bdt_reg_wt 0.05 --edt_reg_wt 0.1 --kp_loss_wt 10. --deform_reg_wt 1e-4  --display_freq 50 --print_freq 50  --tex_loss_wt 1. --init_camera_emb --optimize_deform



/content/acfm/multiframe
Number of videos for  horse - 0
Training samples (frames): 0
Num frames: 2
Num frames: 2
Traceback (most recent call last):
  File "main.py", line 963, in <module>
    app.run(main)
  File "/usr/local/lib/python3.8/site-packages/absl/app.py", line 300, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.8/site-packages/absl/app.py", line 251, in _run_main
    sys.exit(main(argv))
  File "main.py", line 958, in main
    trainer.init_training()
  File "/content/acfm/multiframe/nnutils/train_utils.py", line 159, in init_training
    self.init_dataset()
  File "main.py", line 309, in init_dataset
    self.dataloader = DataLoader(self.dataset, opts.batch_size, drop_last=True, shuffle=True,
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/dataloader.py", line 224, in __init__
    sampler = RandomSampler(dataset, generator=generator)
  File "/usr/local/lib/python3.8/site-packages/torch/utils/data/sampler.py", line 95, in __init__
    raise V